# Applied Data Science Capstone Project

## Segmenting and Clustering Neighborhoods in Toronto

In [2]:
import pandas as pd
import numpy as np

**1. Build pandas dataframe of Toronto neighborhoods retrieving data from a Wikipedia page**

In [4]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


**2. Remove the observations where the borough is not assigned**

In [20]:
df = df[df['Borough'] != 'Not assigned'].reset_index(drop=True)
df.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


**3. Replace unassigned neighborhoods with the associated boroughs**

In [17]:
df['Neighborhood'] = np.where(df['Neighborhood'] == 'Not assigned', df['Borough'], df['Neighborhood'])
df.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


**4. Replace the backlashes ("/") in the Neighborhood column with commas (",") and remove the space before the commas**

In [35]:
df['Neighborhood'] = df['Neighborhood'].str.replace(' /',',')
df.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


**5. Retrieve the shape of the pandas dataframe**

In [38]:
print(df.shape)

(103, 3)
